In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, AssistantAgent
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import AzureOpenAIChatCompletionClient

import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

In [3]:
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

/var/folders/v1/cv54pg7n7rdfjb_37_f20cd00000gn/T/ipykernel_22768/3143287853.py:1: DeprecationWarning: autogen_ext.models.AzureOpenAIChatCompletionClient moved to autogen_ext.models.openai.AzureOpenAIChatCompletionClient. This alias will be removed in 0.4.0.
  client = AzureOpenAIChatCompletionClient(
/Users/sjuratovic/repos/autogen-experiments/.venv/lib/python3.10/site-packages/autogen_ext/models/openai/_openai_client.py:1061: UserWarning: Previous behavior of using the model name as the deployment name is deprecated and will be removed in 0.4. Please specify azure_deployment
  client = _azure_openai_client_from_config(copied_args)


In [4]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed tfinal travel plan. You must ensure th b at the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

In [5]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent],
    termination_condition=termination
)

In [6]:
result = await group_chat.run(task="Plan a 3 day trip to Paris.")

In [7]:
from IPython.display import Markdown, display

total_prompt_tokens = 0
total_completion_tokens = 0

for index, message in enumerate(result.messages, start=1):
    display(Markdown('<hr>'))
    text = f"## Step: {index} (output from {message.source})"
    display(Markdown(text))

    if message.models_usage:
        print(f'Prompt Tokens: {message.models_usage.prompt_tokens}')
        print(f'Completion Tokens: {message.models_usage.completion_tokens}')
        total_prompt_tokens += message.models_usage.prompt_tokens
        total_completion_tokens += message.models_usage.completion_tokens
    display(Markdown('<hr>'))

    display(Markdown(message.content))

display(Markdown('<hr>'))
print(f'Total Prompt Tokens: {total_prompt_tokens}')
print(f'Total Completion Tokens: {total_completion_tokens}')
print(f'Total Tokens: {total_prompt_tokens + total_completion_tokens}')

<hr>

## Step: 1 (output from user)

<hr>

Plan a 3 day trip to Paris.

<hr>

...

Prompt Tokens: 1232
Completion Tokens: 827


<hr>

Here is a comprehensive 3-day travel plan to Paris that incorporates sightseeing, cultural experiences, local dining, helpful language tips, and practical advice to make your visit enjoyable:

---

### Day 1: Classic Paris

**Morning:**
- **Eiffel Tower**: Begin your day with a visit to the Eiffel Tower. Arrive early to avoid the crowds and enjoy a panoramic view of Paris. Purchase your tickets in advance.
  - **Language Tip**: Greet staff with “Bonjour” and remember to say “Merci” after your visit.

**Afternoon:**
- **Seine River Cruise**: After your Eiffel Tower visit, relax with a scenic Seine River cruise. You'll pass by iconic landmarks, including the Notre-Dame Cathedral and the Louvre.
- **Lunch**: Enjoy a leisurely lunch at a café in the Latin Quarter or near Saint-Germain-des-Prés. Taste some French specialties like quiche or a croque-monsieur.
  - **Cultural Tip**: Begin your meal interactions with "Bonjour" and ask for "L'addition, s'il vous plaît" when ready to leave.

**Evening:**
- **Champs-Élysées & Arc de Triomphe**: Stroll along the famous avenue towards the Arc de Triomphe. Consider climbing to the top for sunset views of the lively Champs-Élysées.
- **Dinner in the Marais**: Enjoy dinner at a traditional Parisian bistro in the bustling Marais district.

---

### Day 2: Art and Culture

**Morning:**
- **Louvre Museum**: Spend the morning at the Louvre. Arrive early and pre-purchase a museum pass to skip the lines. Make sure to see the Mona Lisa and other famed masterpieces.
  - **Language Tip**: Familiarize yourself with words like "Entrée" (Entrance).

**Afternoon:**
- **Lunch in Montmartre**: Travel to Montmartre for lunch. Explore the artsy district and visit the Sacré-Cœur Basilica, where you’ll have an amazing view of Paris.
- **Musée d'Orsay**: Dedicate your afternoon to the Musée d'Orsay, featuring works by Monet, Van Gogh, and other renowned artists.

**Evening:**
- **Moulin Rouge Show or Evening Stroll**: Depending on your interest, consider attending a vibrant show at the Moulin Rouge or enjoy a peaceful evening walk along the Seine.
  - **Cultural Sensitivity Tip**: Parisians appreciate politeness. Start interactions with a greeting such as "Bonsoir" in the evening.

---

### Day 3: Hidden Gems and Relaxation

**Morning:**
- **Le Marais & Bastille**: Spend your morning exploring the boutique shops and art galleries in Le Marais. Learn about the historical significance of the Bastille.
- **Picnic at Luxembourg Gardens**: Buy fresh pastries, wine, and cheese from a nearby market and enjoy a relaxed picnic in the Luxembourg Gardens.

**Afternoon:**
- **Île de la Cité & Notre-Dame**: Visit the Île de la Cité and admire the exterior of Notre-Dame Cathedral. Head to Sainte-Chapelle to witness its remarkable stained glass windows.

**Evening:**
- **Dinner in the Latin Quarter**: Choose a cozy brasserie or traditional French restaurant in the Latin Quarter for your farewell dinner.
- **Eiffel Tower at Night**: Conclude your trip with a view of the Eiffel Tower illuminating the night sky as it sparkles every hour.

---

**Additional Tips:**

- **Transport**: Consider purchasing a Paris Visite pass for unlimited travel on public transportation or opt for a carnet of metro tickets for flexibility.
- **Navigation and Language Apps**: Use Google Maps and language translation apps to assist with directions and communication.
- **Cultural Awareness**: Always begin conversations with polite greetings and respect local customs.

Enjoy your magical stay in Paris while embracing the art, history, and romance that the city has to offer. Bon voyage!

TERMINATE

<hr>

Total Prompt Tokens: 2975
Total Completion Tokens: 1948
Total Tokens: 4923
